In [1]:
import pandas as pd
import numpy as np
import mne

from functions.bad_channels import bad_channels
from _parameters import *

In [6]:
def preprocess(s: int, file: str):
    
    # Read data
    ses1, ses2 = file

    raw1 = mne.io.read_raw_bdf(ses1, preload=True)
    raw2 = mne.io.read_raw_bdf(ses2, preload=True)

    raw = mne.concatenate_raws([raw1, raw2])

    # Re-reference
    raw.set_eeg_reference(['EXG1', 'EXG2'])

    # Bipolar emg/eog
    raw = mne.set_bipolar_reference(raw, 'EXG3', 'EXG4', 'EOG',
                                    copy = False, drop_refs = True)
    raw = mne.set_bipolar_reference(raw, 'EXG5', 'EXG6', 'EMGL',
                                    copy = False, drop_refs = True)
    raw = mne.set_bipolar_reference(raw, 'EXG7', 'EXG8', 'EMGR',
                                    copy = False, drop_refs = True)

    raw.set_channel_types({'EOG':'eog'})
    raw.set_channel_types({'EMGL':'emg', 'EMGR':'emg'})

    # Drop and interpolate
    raw.drop_channels(drop_chan)

    raw = bad_channels(raw, picks = "eeg", sig_lev=0.05)
    raw.interpolate_bads()

    # High pass filter EMG
    raw.filter(l_freq = 60, h_freq = None, picks = ['EMGL','EMGR'])

    # Resample
    raw.resample(200)

    # Montage
    biosemi64 = mne.channels.make_standard_montage('biosemi64')
    eeg_map = dict(zip(raw.info.ch_names[:64],biosemi64.ch_names))

    raw.rename_channels(eeg_map)
    raw.set_montage(biosemi64)

    # Save clean raw
    raw_fname = dirs['home'] + dirs['raw'] + '/raw_s' + str(s) + '.fif'
    
    raw.save(raw_fname, overwrite = True)

    return raw

In [5]:
s = 1; file = files['eeg'][0]

raw = preprocess(s, file)

raw.info

Extracting EDF parameters from /Users/rosenasrawi/Documents/VU PhD/Projects/rn6 - Temporal action/Data/lab/EEG/rn6_s1a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3284991  =      0.000 ...  3207.999 secs...
Extracting EDF parameters from /Users/rosenasrawi/Documents/VU PhD/Projects/rn6 - Temporal action/Data/lab/EEG/rn6_s1b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3087359  =      0.000 ...  3014.999 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=6372352
    Range : 0 ... 6372351 =      0.000 ...  6222.999 secs
Ready.
Added the following bipolar channels:
EOG
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=6372352
    Range : 0 ... 6372351 =      0.000 ...  6222.999

/var/folders/5k/8wyp03753yj_s079b4fxjc780000gn/T/ipykernel_3192/376141468.py:29: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads()


Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
3841 events found
Event IDs: [    1     2     3     4     5     6     7     8    11    12    13    14
    15    16    17    18    21    22    23    24    25    26    27    28
    31    33    35    36    37    38    41    42    44    45    47    48
    51    52    53    54    55    56    57    58    61    62    64    65
    67    68    71    73    75    76    77    78 65536]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
3841 events found
Event IDs: [    1     2     3     4     5     6     7     8    11    12    13    14
    15    16    17    18    21    22    23    24    25    26    27    28
    31    33    35    36    37    38    41    42    44    45    47    48
    51    52    53    54    55    56    57 

/var/folders/5k/8wyp03753yj_s079b4fxjc780000gn/T/ipykernel_3192/376141468.py:47: RuntimeWarning: This filename (/Users/rosenasrawi/Documents/VU PhD/Projects/rn6 - Temporal action/Data/processed/raw/raw_s1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(raw_fname, overwrite = True)


Closing /Users/rosenasrawi/Documents/VU PhD/Projects/rn6 - Temporal action/Data/processed/raw/raw_s1.fif
[done]


<Info | 9 non-empty values
 bads: []
 ch_names: Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, ...
 chs: 64 EEG, 1 Stimulus, 1 EOG, 2 EMG
 custom_ref_applied: True
 dig: 67 items (3 Cardinal, 64 EEG)
 highpass: 0.0 Hz
 lowpass: 100.0 Hz
 meas_date: 2023-02-23 15:04:13 UTC
 nchan: 68
 projs: []
 sfreq: 200.0 Hz
>